# Sujet de mini-projet (2019): Cooperative Path-Finding

## Environnement
Vous utiliserez le module `PySpriteWorld` qui élabore pygame et permet de manipuler simplement des personnages, cartes, et autres objets à l'écran. Ce module a été développé par Yann Chevaleyre. Une version plus complète se trouve [ici](https://github.com/yannche/pySpriteWorld), mais la version disponible dans ce répertoire suffit a priori pour faire tout ce dont vous avez besoin.

Notez que vous pourrez ensuite éditer vos propres cartes à l'aide de l'éditeur [Tiled](http://www.mapeditor.org/), et exporter ces cartes au format `.json`.
Il faut utiliser au moins trois calques lors de la création de votre carte: 
* un calque **joueur**, où seront les personnages 
* un calque **ramassable**, qui contient les objets que les personnages peuvent ramasser
* un calque **obstacles**, pour les murs, les arbres, etc.

**Note**: on fait ici l'hypothèse que toutes les informations (positions des agents et des trésors) sont disponibles pour tous les agents (i.e. on ne se pose pas de problème de communication)


Dans ce projet, on considère à présent en compétition que plusieurs personnages doivent chacun atteindre un objectif qui leur propre (une fiole donnée). On souhaite éviter les collisions entre personnages, ce qui signifie que l'on souhaite que les agents possèdent des algorithmes qui leurs permettent de s'éviter. Sont considérées comme collisions les situations où les personnages:

* se trouvent au même moment sur la même case, ou bien
* deux personnages se "croisent"

**Note**: pour utiliser les testes il faut que le notebook soit dans le meme dossier que les fichier .py

# Strategie Path Slicing

* Cette Strategie consiste calculer le plus court chemin de chaque agent vers sa fiole et a modifier le chemin emprunté par l'agent lorsque celui-ci détecte une collision  
* Pour calculer le plus court chemin d'un agent vers sa fiole on utilise l'algorithme A* en utilisant la distance de Manhattan comme heuristique
* L'algorithme A* garanti de ne pas avoir de colision avec les murs

## Fonctionement

Au demarage chaque agent (class PlayerPath) calcule son chemin vers sa fiole en utilisant l'algorithme A*. Une fois les chemins calculés on parcours les chemins des agents (class PathSlicingStrategy) et a chaque tour, s'il y a colision l'agent qui a detecté la colision recalcule son chemin en ajoutent un chemin a partir de la case avant la colision vers la case apres la colision.

## Testes

In [2]:
import mainTest

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
m = mainTest.initTest('carte1')
m.pathSlicingStrategyTest()

In [3]:
m = mainTest.initTest('carte2')
m.pathSlicingStrategyTest()

Init states: [(1, 3), (4, 16), (18, 1)]
Goal states: [(6, 7), (12, 6), (19, 8)]
<Group(3 sprites)>
Cooperative Basic Strategy : done in  20  moves


In [4]:
m = mainTest.initTest('carte3')
m.pathSlicingStrategyTest()

Init states: [(2, 4), (4, 16), (18, 1)]
Goal states: [(4, 9), (9, 13), (12, 6)]
<Group(3 sprites)>
Cooperative Basic Strategy : done in  18  moves


In [7]:
m = mainTest.initTest('carte4',True)
m.pathSlicingStrategyTest()

Init states: [(2, 4), (4, 16), (18, 1)]
Goal states: [(4, 9), (9, 13), (12, 6)]
<Group(3 sprites)>
Path Slicing Strategy : done in  16  moves


In [ ]:
m = mainTest.initTest('carte5')
m.pathSlicingStrategyTest()

le prochain teste montre que parfois le Path Slicing nous donne un chemin mais qui n'est pas le plus court

In [ ]:
m = mainTest.initTest('carte6', True)
m.pathSlicingStrategyTest()

# Strategie Coopérative de Base

* Cette consiste calculer le plus court chemin de chaque agent vers sa fiole et a diviser les agents par groupes de tels sorte que chaque agent d'un groupe ne croise jamais un autre agent du meme groupe lors d'une excecution simultané des agents

## Fonctionement

* Tout comme le Path Slicing dans une premier temps chaque agent calcule son chemin vers sa fiole en utilisant l'algorithme A*. Une fois les chemins calculés on separe les agents par groupes (class CoopBaseStrategy) 
* Chaque groupe corespond a un objet (class Group) qui contient en plus de la liste des agents un liste des cases parcouru par chaque agent pour pouvoir detetcter si un le chemin d'un autre agent crois les chemins des agents de ce  groupe (detection de croisement naïve)
* Une fois les groupes formé on fait bougé les agents, lorsque tout les agents d'un groupe on attient leur objectif on passe a l'autre groupe en recaculant le chemin des autres agent en prennant en compte que les agents sur les fioles sont des murs (ce qui peut crée de nouveaux groupes)
* la strategie utilisé pour les decoupages en groupe est celle du premier venu premier servie en d'autres mots on parcours les agents sequentielement sans preference selon la taille du chemin ou autre chose 


## Testes

In [ ]:
m = mainTest.initTest('carte1')
m.coopBaseStrategyTest()

In [ ]:
m = mainTest.initTest('carte2')
m.coopBaseStrategyTest()

In [ ]:
m = mainTest.initTest('carte4', True)
m.coopBaseStrategyTest()

In [ ]:
m = mainTest.initTest('carte5')
m.coopBaseStrategyTest()

avec cette strategie les deux agents on un chemin le plus court mais on perd en nombre de tour car la stategie cherche un seul plus court chemin et non tout les plus courts

In [ ]:
m = mainTest.initTest('carte6')
m.coopBaseStrategyTest()

# Stratégie Coopérative Avancée

##### Les deux Stratégie precedente ont chaqu'une des lacunes :
* Le Path Slicing peut obliger un agent a faire un plus long chemin car le nouveau path est calculé d'un point avant la colision vers un point apres la colision
* La Strategie Coopérative de Base donne parfois un temps d'execution plus grand que l'excecution la plus optimal car il ne prend par en compte le moment ou les agents peuvent se croiser (time) mais prend en compte juste qu'une colision soit possible


Pour la stratégie coopérative avancée on utilise une table de réservation spatio-temporelle où l'on va stocker des triplets qui correspondent à la position à un instant donné et comma ça lors de la planification du chemin chaque agent sait ou les autres agents seront a l'instant t

## Fonctionement

* Chaque agent calcule le plus court chemin vers la fiole on utilisant l'algorithme A* spatio-temporelle qui va considere que les emplacements reservé par les autres agents au moment t sont des murs et va les eviters
* Pour calculer le plus court chemin d'un agent vers sa fiole on utilise l'algorithme A* spatio-temporelle utilisant la vraie distance comme heuristique obtenu en utilisant l'algorithme A* de la fiole vers la position de l'agent

## Testes

In [ ]:
m = mainTest.initTest('carte1')
m.coopAdvStrategyTest()

In [ ]:
m = mainTest.initTest('carte2')
m.coopAdvStrategyTest()

In [ ]:
m = mainTest.initTest('carte3')
m.coopAdvStrategyTest()

In [ ]:
m = mainTest.initTest('carte4')
m.coopAdvStrategyTest()

In [ ]:
m = mainTest.initTest('carte11')
m.coopAdvStrategyTest()

In [ ]:
m = mainTest.initTest('carte7', True)
m.coopAdvStrategyTest()

In [ ]:
m = mainTest.initTest('carte9', True)
m.coopAdvStrategyTest()

# Comparaison

In [ ]:
mainTest.testAll('carte1')

In [ ]:
mainTest.testAll('carte2')

In [ ]:
mainTest.testAll('carte3')

In [ ]:
mainTest.testAll('carte4')

In [ ]:
mainTest.testAll('carte5')

In [ ]:
mainTest.testAll('carte6', True)

In [ ]:
mainTest.testAll('carte7', True)

In [ ]:
mainTest.testAll('carte8', True)

In [ ]:
mainTest.testAll('carte9', True)

In [ ]:
mainTest.testAll('carte11')

# Conclusion

D'apres les testes qu'on a fait on remarque que la Stratégie Coopérative Avancée a toujours le meilleur score selon le nombre d'iteration mais elle prend plus de temps de calcule que les autres (ce qui peut etre amelioré en ajoutant une profondeur de recherche et une priorité a chaque agent ) et elle ne résout pas les cartes impossibles comme la 8, la Stratégie Coopérative de Base donne presque toujours le plus grand nombre d'iteration car les agents sont trop longtemps immobiles mais reste meilleure que la strategie de Path Slicing dans certain cas, et au final pour le Path Slicing on a remarqué que parfois elle fait faire des detours ce qui nous donne un resultat qui n'est pas optimal.